In [6]:
import pandas as pd
import re

In [7]:

df = pd.read_csv("minutes_stage1.csv", encoding="utf-8-sig")

# BODY만 추출
df_body = df[df["text_type"] == "BODY"].copy()

print(df_body.shape)
df_body.head()


(265, 6)


,meeting_date,release_date,text_date,text_type,text,source_type
1,2011-11-24,2012-01-13,2012-01-13,BODY,먼저 증권대차제도에 대하여 논의가 이루어졌음 일부 위원은 향후 증권대여를 활용해야 ...,minutes
4,2011-12-08,2012-01-25,2012-01-25,BODY,국내외 경제동향 과 관련하여 일부 위원은 경제성장률이 올해 뿐만 아니라 내년까지 연...,minutes
9,2011-12-22,2012-02-07,2012-02-07,BODY,일부 위원은 당행은 그동안 금융시장 여건 개선추세를 보아가며 글로벌 금융위기 대응과...,minutes
12,2011-12-29,2012-02-14,2012-02-14,BODY,먼저 관련부서에서는 본 의안과 관련하여 그 동안 위원들이 위원협의회 등에 서 밝힌 ...,minutes
16,2012-01-13,2012-02-28,2012-02-28,BODY,국내외 경제동향 과 관련하여 일부 위원은 최근 건설투자 등 일부 경제지표 가 지난해...,minutes
